# Noise ceilings for connectivity models
This notebook explains the different noise ceilings we can use for individual and group connectivity models. The noise ceiling is trying to capture the expected performance on a specific set of test data, if a) connectivity was perfectly linear and b) we new the true connectivity weights $\mathbf{W}$. 
The individual noise ceiling is the performance if we knew the individual weights $\mathbf{W}_s$, the group noise ceiling is the performance if we knew the true group mean $\mathbf{W}$. 

###  1.Definitions 
Let $\mathbf{Y}_i$ be the cerebellar data for partition (run, session, half) $i$. 
Also let $\mathbf{X}_i$ the cortical data for that partition - and $\mathbf{W}$ the connectivity weights for the model under question.  

### 2. Individual noise ceiling based on cerebellar data only
One limiting factor for the prediction performance in the measurement noise on the cerebellar data  (the prediction target). If each session is composed of the true activity patterns $\mathbf{U}$ and some noise $\boldsymbol{\epsilon}$: 

$$\mathbf{Y}_1 = \mathbf{U} +  \boldsymbol{\epsilon}_1$$ 

and $vU$ signifies the variance of the true signal, whereas $vE$ is the variance of the noise, then the correlation between the two sessions is 

$$r(\mathbf{Y}_1,\mathbf{Y}_2) = \frac{vU}{\sqrt{vU+vE}\sqrt{vU+vE}}$$ 

In turn the noise ceiling (best performance of true model, i.e. the correlation between true signal and measured signal) is: 

$$r_{ceil1} = r(\mathbf{U},\mathbf{Y}_i) = \frac{vU}{\sqrt{(vU)}\sqrt{(vU+vE)}}=\frac{\sqrt{vU}}{\sqrt{vU+vE}}=\sqrt{r(\mathbf{Y}_1,\mathbf{Y}_2)}$$

### 3. Individual noise ceiling based both on cerebellar and cortical data
The noise ceiling based on cerebellar data only is somewhat optimistic, as it demands that we know both the true connectivity weights $\mathbf(W)$, as well we the true cortical activation states $\mathbf{X}$. So, we need to consider the fact that the prediction $\mathbf{W} \mathbf{X}_i$ has noise variance $vP$, which we can estimate by calculating the the correlation: 

$$r(\mathbf{W}\mathbf{Y}_1,\mathbf{W}\mathbf{Y}_2) = \frac{vU}{\sqrt{vU+vP}\sqrt{vU+vP}}$$ 

then the adjusted noise ceiling becomes 

$$r_{ceil2} = r(\mathbf{X}_i \mathbf{W},\mathbf{Y}_i) = \frac{vU}{\sqrt{(vU+vP)}\sqrt{(vU+vE)}}=\sqrt{r(\mathbf{Y}_1,\mathbf{Y}_2) r(\mathbf{W}\mathbf{Y}_1,\mathbf{W}\mathbf{Y}_2)}$$

Since we don't know the true weights, we can only use the weights from the current model - so the noise ceiling becomes model dependent.

### 4. Code implementation

The module `evaluation.py` has a function `calculate_reliability` which computes the reliability between the sessions - so the snippet of `eval_models` in the `run` module gets you the reliabilities for the noise ceiling:  

In [ ]:
import cortico_cereb_connectivity.evaluation as ev
import numpy as np

In [ ]:
noise_Y_R, _, noise_Y_R2, _ = ev.calculate_reliability(Y,T) # reliability  for cerebellum
noise_X_R, _, noise_X_R2, _ = ev.calculate_reliability(Ypred,T) # reliability for cortex

These values are stored for each model + ROI in the data frame, returned by `eval_models`. From these values we can get the noise ceiling: 

In [ ]:
noise_1 = np.sqrt(noise_Y_R)

noise_2 = np.sqrt(noise_X_R * noise_Y_R) # model specific noise ceilung 